# CART Implementation

Import Packages

In [17]:
import math
import pandas as pd
import numpy as np
import requests
from io import StringIO
from gurobipy import *
from sklearn import tree
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from ucimlrepo import fetch_ucirepo as fetc
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


Import Data and Check Data Status

In [ ]:
rice = fetc(id=545)
X = rice.data.features 
y = rice.data.targets
df = pd.DataFrame(X, columns=rice.data.feature_names)
df['target'] = y

#Change non numeric columns to numeric
column_names = ["Area", "Perimeter", "Major_Axis_Length", "Minor_Axis_Length","Eccentricity", "Convex_Area", "Extent", "target"]
non_numeric_cols = df.drop(['target'],axis=1).select_dtypes(exclude=[np.number]).columns
if not non_numeric_cols.empty:
    for col in non_numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
for col in column_names[:len(column_names)-1]:
    df[col] = StandardScaler().fit_transform(df[[col]])
    
for col in column_names[:len(column_names)-1]:
    df[col] = MinMaxScaler().fit_transform(df[[col]])

#Convert target column to numeric
le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])
        
df.info()
#df.isnull().sum()
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3810 non-null   float64
 1   Perimeter          3810 non-null   float64
 2   Major_Axis_Length  3810 non-null   float64
 3   Minor_Axis_Length  3810 non-null   float64
 4   Eccentricity       3810 non-null   float64
 5   Convex_Area        3810 non-null   float64
 6   Extent             3810 non-null   float64
 7   target             3810 non-null   int64  
dtypes: float64(7), int64(1)
memory usage: 238.3 KB
       Area  Perimeter  Major_Axis_Length  Minor_Axis_Length  Eccentricity  \
0  0.675937   0.879232           0.901216           0.532417      0.888011   
1  0.625330   0.714095           0.648087           0.670663      0.691980   
2  0.623394   0.750066           0.734349           0.588124      0.789846   
3  0.495071   0.524136           0.512800         

Split Data into Training and Testing Sets

In [22]:
X = df.drop(['target'], axis=1)
y = df['target']
print("Total number of Class: " + str(len(np.unique(y))) + " which are: " + str(np.unique(y)))
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

Total number of Class: 2 which are: [0 1]


Decision Tree

In [23]:
def clf(x_train, y_train, x_test, y_test,K):
    clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=K, random_state=42)
    clf_gini.fit(x_train, y_train)
    y_pred_gini = clf_gini.predict(x_test)
    y_pred_gini_train = clf_gini.predict(x_train)
    print('Testing Set Accuracy Score: ', accuracy_score(y_test, y_pred_gini), 'Training Set Accuracy Score:',accuracy_score(y_train, y_pred_gini_train))

clf(x_train, y_train, x_test, y_test,3)

Testing Set Accuracy Score:  0.9300699300699301 Training Set Accuracy Score: 0.9286596664607782


Tree Structure: Will be Used for Warmstart of OCT

In [24]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
clf.fit(x_train, y_train)

"""
The following is for warm start in OCT
"""
feature = clf.tree_.feature
threshold = clf.tree_.threshold
feature_indices = []
threshold_indices = []
feature_indices_leaf =[]
threshold_indices_leaf = []
for i in range(len(feature)):
    if feature[i] != -2 and feature[i+1] != -2:
        feature_indices.append(int(feature[i]))
        threshold_indices.append(float(threshold[i]))
    elif feature[i] != -2 and feature[i+1] == -2:
        feature_indices_leaf.append(int(feature[i]))
        threshold_indices_leaf.append(float(threshold[i]))
    else:
        continue
feature_indices.extend(feature_indices_leaf)
threshold_indices.extend(threshold_indices_leaf)
print("Feature Indices: ", feature_indices, "Threshold Indices:", threshold_indices)
#leaf nodes
#branch nodes
print(np.sum(clf.tree_.feature == -2))
internal_nodes = np.sum(clf.tree_.feature != -2)
total_nodes = clf.tree_.node_count
print(f"Internal nodes: {internal_nodes}, Total nodes: {total_nodes}")
        

Feature Indices:  [2, 2, 1, 2, 2, 0, 2] Threshold Indices: [0.5003375709056854, 0.4399276226758957, 0.5745012164115906, 0.3837266117334366, 0.4810698330402374, 0.4023939371109009, 0.5641956627368927]
8
Internal nodes: 7, Total nodes: 15


In [66]:
total_nodes = clf.tree_.node_count
leaf_nodes = round(total_nodes / 2)
branch_nodes = total_nodes // 2
print(f"Leaf nodes: {leaf_nodes}, Branch nodes: {branch_nodes}")

Leaf nodes: 8, Branch nodes: 7


# OCT Implementation

Import Packages

In [9]:
import gurobipy as gp
from gurobipy import GRB

In [10]:
m = gp.Model('OCT')

Construct Constraints

In [11]:
"""
training data (X,Y)=(x_i,y_i), i=1,...,n; x_i in R^p and normalized to [0,1]^p, y_i in {1,...,K}
-n = # obervations
-p = # features
-K = # class label

-p(t) = parent nodes of node t
-A_L(t) = {left_branch ancestors of t} = {t if tmod2=0 and t/2 recursively gives the set}
-A_R(t) = {right_branch ancestors of t} = {t if (t-1)mod2=0 and (t-1)/2 recursively gives the set}


-D = max depth
-T = 2^(D+1)-1 = max # nodes
-TB = branch nodes = left nodes = {1,...,floor(T/2)} applies split is ax<b 
-TL = leaf = {floor(T/2)+1,...,T} make class prediction
-?a_t in R^p
-?b_t in R
-p = # features

-d_t = 1{node t applies a split}
-sum a_jt = d_t, j=1,...p, t in TB
-0 <= b_t <= d_t, t in TB
-a_jt in {0,1}, j=1,...p, t in TB


-d_t <= d_p(t), t in TB/{1}

-z_it = 1{x_i in node t}
-l_t = 1{leaf t contains any point}
-z_it <= l_t, t in TB
-sum(z_it) >= N_min*l_t for i=1,...,n, t in TB
-sum(z_it)=1 for t in TB, i=1,...,n

-x_j^i = ith largest value in feature j
-epsilon_j = min{x_j^(i+1)-x_j^i, i=1,...,n}
-epsilon_max = max{epsilon_j} wrt j
-epsilon  = {epsilon_1,...,epsilon_p}
-a_m(x_t + epsilon) <= b_m +(1+epsilon_max)(1-z_it), i=1,...,n, for all t in TB, for all m in A_L(t)
-a_m*x_i >= b_m - (1-z_it), i=1,...,n, for all t in TB, for all m in A_R(t)

-Y_ik = {1 if y_i=k, -1 otherwise}, k=1,...,K, i=1,...n
N_kt = 0.5*sum((1+Y_ik)z_it) for i=1,...,n; k=1,...K, t in TL
N_t = sum(z_it) for i=1,...,n; t in TL
c_t = argmax{N_kt} wrt k
c_kt = 1{c_t = k}
sum(c_kt) = l_t wrt k

L_t = N_t - max{N_kt} wrt k = min{N_t - N_kt} wrt k
L_t >= N_t - N_kt - n(1-c_kt), k=1,...K, t in TL
L_t <= N_t - N_kt + n*c_kt, k=1,...K, t in TL
L_t >= 0

L^ = baseline accuracy = #{most popular class}/n

objective: min (1/L^)sum(L_t) for t in TL + alpha*sum(d_t) for t in TB

"""

'\ntraining data (X,Y)=(x_i,y_i), i=1,...,n; x_i in R^p and normalized to [0,1]^p, y_i in {1,...,K}\n-n = # obervations\n-p = # features\n-K = # class label\n\n-p(t) = parent nodes of node t\n-A_L(t) = {left_branch ancestors of t} = {t if tmod2=0 and t/2 recursively gives the set}\n-A_R(t) = {right_branch ancestors of t} = {t if (t-1)mod2=0 and (t-1)/2 recursively gives the set}\n\n\n-D = max depth\n-T = 2^(D+1)-1 = max # nodes\n-TB = branch nodes = left nodes = {1,...,floor(T/2)} applies split is ax<b \n-TL = leaf = {floor(T/2)+1,...,T} make class prediction\n-?a_t in R^p\n-?b_t in R\n-p = # features\n\n-d_t = 1{node t applies a split}\n-sum a_jt = d_t, j=1,...p, t in TB\n-0 <= b_t <= d_t, t in TB\n-a_jt in {0,1}, j=1,...p, t in TB\n\n\n-d_t <= d_p(t), t in TB/{1}\n\n-z_it = 1{x_i in node t}\n-l_t = 1{leaf t contains any point}\n-z_it <= l_t, t in TB\n-sum(z_it) >= N_min*l_t for i=1,...,n, t in TB\n-sum(z_it)=1 for t in TB, i=1,...,n\n\n-x_j^i = ith largest value in feature j\n-epsilo

Predetermined Variables

In [85]:
n= df.shape[0] # number of observations
p = df.drop(['target'],axis=1).shape[1] # number of features
K = len(np.unique(df['target'])) # number of classes
D = 3 # max depth
T = 2**(D+1)-1 # max number of nodes
L_hat = df['target'].value_counts().max()/n # baseline accuracy

N_min = math.floor(n*0.05)

#Epsilon
epsilon=[]
for j in range(p):
    x_j = df.iloc[:,j].tolist()
    x_j.sort()  
    e=[]
    for i in range(n-1):
        if x_j[i+1]!=x_j[i]:
            e.append(x_j[i+1] - x_j[i])
    epsilon.append(min(e))
epsilon_max = max(epsilon)

#Y Matrix
Y = np.zeros([n,K], dtype = int) - 1 # Y_ik
Y[df.index,  y] = 1  #based on the sample code, the [x,y] x is the features index, y is the class index
print(epsilon)

#label_to_binary = {'Cammeo': 1, 'Osmancik': 0}
#y_binary = np.array([label_to_binary[label] for label in y])
#print("y binary: " + str(y_binary)) 
#Y_ik = np.zeros([n,K], dtype = int) - 1
#Y_ik[df.index,  y] = 1  #based on the sample code, the [x,y] x is the features index, y is the class index

[8.801267382496647e-05, 5.1572260014176585e-06, 3.253471011488429e-07, 1.5892507770898234e-07, 3.454851569273387e-07, 8.790436005612356e-05, 1.649998595532054e-07]


Predetermined Sets (Note that index starts from 1)

In [86]:
parent = []
for t in range(1,T+1):
    if t%2==0:
        parent.append(t//2)
    else:
        parent.append((t-1)//2)
print(parent)

left_ancestors = []
right_ancestors = []
for t in range(1,T+1):
    la_t =[]
    ra_t =[]
    tau=t
    while tau>1:
        pt = tau//2
        if tau % 2 == 0: #if t is even, then its parent is a left ancestor, else is a right ancestor
            la_t.append(pt)
        else:
            ra_t.append(pt)
        tau = pt
    left_ancestors.append(la_t)
    right_ancestors.append(ra_t)

TB = list(range(1,math.floor((T+1)/2)))  #Branch nodes
TL = list(range(math.floor((T+1)/2),T+1)) #Leaf nodes
TT = list(range(1,T+1)) #total nodes
print("Branch nodes: ", TB)
print("Leaf nodes: ", TL)
left_ancestors
right_ancestors

[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7]
Branch nodes:  [1, 2, 3, 4, 5, 6, 7]
Leaf nodes:  [8, 9, 10, 11, 12, 13, 14, 15]


[[],
 [],
 [1],
 [],
 [2],
 [1],
 [3, 1],
 [],
 [4],
 [2],
 [5, 2],
 [1],
 [6, 1],
 [3, 1],
 [7, 3, 1]]

Variables and Constraints

Updated Variables and Constraints

In [188]:
a = m.addVars(p,TB, vtype=GRB.BINARY, name="a_t") #dim |TB|xp
b = m.addVars(TB, vtype=GRB.CONTINUOUS, lb = 0, ub = 1, name="b_t") #dim |TB| ###
d = m.addVars(TB, vtype=GRB.BINARY, name="d_t") #dim |TB|
z = m.addVars(n, TL, vtype=GRB.BINARY, name="z") #dim nx|TL|
l = m.addVars(TL, vtype=GRB.BINARY, name="l_t") #dim |TL|
Nk = m.addVars(K, TL, vtype=GRB.INTEGER,name="N_kt") #dim Kx|TL|
N = m.addVars(TL, vtype=GRB.INTEGER, name="N_t") #dim |TL|
ck = m.addVars(K, TL, vtype=GRB.BINARY, name="c_kt")
# c = m.addVars(TL, vtype=GRB.INTEGER, name="c_t") #dim |TL|
#c_bin = m.addVars(K, TL, vtype=GRB.BINARY, name="c_bin") #dim Kx|TL|
L = m.addVars(TL, name="L_t") 

for t in TB:
    m.addConstr(gp.quicksum(a[i,t] for i in range(p)) == d[t], name="sum_constraint_of_ajt") # sum of ajt = dt
    # m.addConstr(b[t] >= 0, name="bt_constraint_0")         # 0 <= bt
    m.addConstr(b[t] <= d[t], name="bt_constraint_dt")     # bt <= dt

for t in TB[1:]:
    m.addConstr(d[t] <= d[t//2], name="dt_constraint_dp(t)") # dt <= dp(t) 
    
# for t in TB:
#     m.addConstr(d[t] == 1, name="dt_constraint_d(t)") 
    
# for t in TL:
#     m.addConstr(l[t] == 1, name="dt_constraint_l(t)")

for t in TL:
    for i in range(n):
        m.addConstr(z[i, t] <= l[t]) # zit <= lt

for t in TL:
    m.addConstr(gp.quicksum(z[i,t] for i in range(n)) >= N_min*l[t], name="sum_of_zt_constraint_Nmin_lt") # sum of zit >= Nmin*lt
    
for i in range(n):
    m.addConstr(gp.quicksum(z[i,t] for t in TL) == 1, name="sum_of_zi(t)_constraint_1")  # sum sum of zit = 1


for t in TL:
    for k in range(K):
        m.addConstr(Nk[k,t] == 1/2 * gp.quicksum(z[i,t] * (Y[i,k] + 1) for i in range(n))) #Nkt = 1/2(sum of (1+Yik)*zit #may need to be corrected
         
for t in TL:
    m.addConstr(N[t] == gp.quicksum(z[i, t] for i in range(n)))  # Nt = sum of zit

for t in TL:
    m.addConstr(l[t] == gp.quicksum(ck[k, t] for k in range(K))) # sum of ckt = lt
    
for t in TL:
    if len(left_ancestors[t-1]) != 0:
        for la in left_ancestors[t-1]:
            for i in range(n):
                axe = sum([(a[j,la])*(X.iloc[i].tolist()[j] + epsilon[j]) for j in range(p)])
                m.addConstr(axe <= b[la] + (1 + epsilon_max) * (1 - z[i,t]), name="split_structure_constraint_l_i")
                
    if len(right_ancestors[t-1]) != 0:
        for r in right_ancestors[t-1]:
            for i in range(n):
                ade = sum([(a[j,r])*(X.iloc[i].tolist()[j]) for j in range(p)])
                m.addConstr(ade >= b[r] - (1 - z[i,t]), name="split_structure_constraint_r_i")

#for t in TL:
    #m.addConstr(gp.quicksum(c_bin[k, t] for k in range(K)) == 1, name=f"unique_selection_t")
#for t in TL:
    #for k in range(K):
        #m.addConstr(c[t] >= k - (1 - c_bin[k, t]) * K, name=f"select_max_kt")
        #m.addConstr(c[t] <= k + (1 - c_bin[k, t])* K, name=f"select_min_kt")
        
for t in TL:
    for k in range(K):
        m.addConstr(L[t] >= N[t] - Nk[k,t] - n*(1-ck[k,t]), name="Lt_constraint1") #Lt ≤ Nt − Nkt + n(1-ckt) 
        m.addConstr(L[t] <= N[t] - Nk[k,t] + n*ck[k,t], name="Lt_constraint2")  #Lt ≤ Nt − Nkt + n*ckt 
    m.addConstr(L[t] >= 0, name="Lt_constraint3") #Lt ≥ 0

In [189]:
l

{8: <gurobi.Var *Awaiting Model Update*>,
 9: <gurobi.Var *Awaiting Model Update*>,
 10: <gurobi.Var *Awaiting Model Update*>,
 11: <gurobi.Var *Awaiting Model Update*>,
 12: <gurobi.Var *Awaiting Model Update*>,
 13: <gurobi.Var *Awaiting Model Update*>,
 14: <gurobi.Var *Awaiting Model Update*>,
 15: <gurobi.Var *Awaiting Model Update*>}

Objective

In [190]:
m.setObjective(gp.quicksum(L[t]/L_hat for t in TL) + 0.5*gp.quicksum(d[t] for t in TB), GRB.MINIMIZE) #minimize L + alpha*sum(d_t) for t in TB

m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 2344963 rows, 581453 columns and 16805688 nonzeros
Model fingerprint: 0x30298d59
Variable types: 285 continuous, 581168 integer (580688 binary)
Coefficient statistics:
  Matrix range     [2e-07, 4e+03]
  Objective range  [5e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 4e+03]
Presolve removed 484949 rows and 103331 columns
Presolve time: 1.29s

Explored 0 nodes (0 simplex iterations) in 2.85 seconds (4.87 work units)
Thread count was 1 (of 22 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [191]:
if m.status == gp.GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    m.computeIIS()
    m.write("infeasible_model.ilp")
    

Model is infeasible. Computing IIS...
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0   2131603         -         0       821         -           0s
        0   2131603         -         0       821         -          29s
        0     34337         -         0        11         -          37s
        0     18250         -         0         8         -          40s
        0      7857         -         0         6         -          45s
        1      5155         -         0         4         -          51s
        2      338

In [193]:
for c in m.getConstrs():
    if c.IISConstr:
        print(f"Infeasible Constraint: {c.constrName}")

for v in m.getVars():
    if v.IISLB or v.IISUB:
        print(f"Infeasible Bound on Variable: {v.varName}")

Infeasible Constraint: sum_of_zi(t)_constraint_1
Infeasible Constraint: split_structure_constraint_l_i
Infeasible Constraint: split_structure_constraint_l_i
Infeasible Constraint: split_structure_constraint_r_i
Infeasible Constraint: split_structure_constraint_r_i
Infeasible Constraint: split_structure_constraint_r_i
Infeasible Constraint: split_structure_constraint_r_i
Infeasible Constraint: split_structure_constraint_r_i
Infeasible Constraint: split_structure_constraint_r_i
Infeasible Bound on Variable: b_t[1]
Infeasible Bound on Variable: b_t[7]


{1: <gurobi.Var b_t[1]>,
 2: <gurobi.Var b_t[2]>,
 3: <gurobi.Var b_t[3]>,
 4: <gurobi.Var b_t[4]>,
 5: <gurobi.Var b_t[5]>,
 6: <gurobi.Var b_t[6]>,
 7: <gurobi.Var b_t[7]>}

In [64]:
left_ancestors


[[],
 [1],
 [],
 [2, 1],
 [1],
 [3],
 [],
 [4, 2, 1],
 [2, 1],
 [5, 1],
 [1],
 [6, 3],
 [3],
 [7],
 []]